In [1]:
import ee
ee.Initialize()

In [2]:
# create a non-geodetic area of interest
aoi = ee.Geometry.Polygon([[
          [-122.75548673853245, 38.0411041965202],
          [-122.75548673853245, 37.64853352930806],
          [-122.11690641626683, 37.64853352930806],
          [-122.11690641626683, 38.0411041965202]]], None, False)

In [3]:
coords_web = aoi.transform('EPSG:3857', 1).coordinates().getInfo()
coords_wgs = aoi.coordinates().getInfo()

In [4]:
# query NED dem for US
dem = ee.Image("USGS/NED").visualize(**{"min": 0, "max": 700})

In [5]:
dem.getMapId()

{'mapid': 'projects/earthengine-legacy/maps/d0d7d18f7cc730af0c35931dda0bad59-41af990e40147365645c95ef98bbd78b',
 'token': '',
 'tile_fetcher': <ee.data.TileFetcher at 0x1ce42dadcc8>,
 'image': <ee.image.Image at 0x1ce42da5708>}

In [6]:
tile_fetcher = dem.getMapId()['tile_fetcher']

In [7]:
url = tile_fetcher.url_format.format(x='${x}', y='${y}', z='${z}')
url

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/d0d7d18f7cc730af0c35931dda0bad59-72c0a00beecabdf416e30dbbb6d2c7ba/tiles/${z}/${x}/${y}'

In [8]:
template =  '''<GDAL_WMS>
    <Service name="TMS">
        <ServerUrl>{url}</ServerUrl>
        <SRS>EPSG:3857</SRS>
    </Service>
    <DataWindow>
        <UpperLeftX>-20037508.34</UpperLeftX>
        <UpperLeftY>20037508.34</UpperLeftY>
        <LowerRightX>20037508.34</LowerRightX>
        <LowerRightY>-20037508.34</LowerRightY>
        <TileLevel>18</TileLevel>
        <TileCountX>1</TileCountX>
        <TileCountY>1</TileCountY>
        <YOrigin>top</YOrigin>
    </DataWindow>
    <Projection>EPSG:3857</Projection>
    <BlockSizeX>256</BlockSizeX>
    <BlockSizeY>256</BlockSizeY>
    <BandsCount>3</BandsCount>
    <Cache>
        <Path>../.gdalwmscache</Path>
        <Extension>.png</Extension>
    </Cache>
</GDAL_WMS>
'''
filled = template.format(url=url)

with open('ee.xml', 'w') as f:
    f.write(filled)

In [9]:
!gdalinfo ee.xml

Driver: WMS/OGC Web Map Service
Files: ee.xml
Size is 67108864, 67108864
Coordinate System is:
PROJCS["WGS 84 / Pseudo-Mercator",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Mercator_1SP"],
    PARAMETER["central_meridian",0],
    PARAMETER["scale_factor",1],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["X",EAST],
    AXIS["Y",NORTH],
    EXTENSION["PROJ4","+proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0 +k=1.0 +units=m +nadgrids=@null +wktext +no_defs"],
    AUTHORITY["EPSG","3857"]]
Origin = (-20037508.339999999850988,20037508.33999999985098

In [10]:
# !gdal_translate ee.xml -scale 1000 -srcwin ${coords_web[0][0][0]} ${coords_web[0][0][1]} 2048 2048 out.png
ll = coords_web[0][0][0], coords_web[0][0][1]
lr = coords_web[0][1][0], coords_web[0][1][1]
ur = coords_web[0][2][0], coords_web[0][2][1]
ul = coords_web[0][3][0], coords_web[0][3][1]

# ll = coords_wgs[0][0][0], coords_wgs[0][0][1]
# lr = coords_wgs[0][1][0], coords_wgs[0][1][1]
# ur = coords_wgs[0][2][0], coords_wgs[0][2][1]
# ul = coords_wgs[0][3][0], coords_wgs[0][3][1]

width = lr[0] - ul[0]
height = ul[1] - lr[1]


In [11]:
!gdal_translate ee.xml -projwin {ul[0]} {ul[1]} {lr[0]} {lr[1]} -outsize 2048 2048 out.png

Input file size is 67108864, 67108864
0...10...20...30...40...50...60...70...80...90...100 - done.
